# Evolutionary approach to the Flow Shop problem 

The Problem
-------------------------
Suppose you have N jobs and M machines. Each job consists in M operations on each machine. The i-th operation is executed on the i-th machine and each operation could start only if the previous have already finished. Jobs can be executed in any order. Jobs order could differ depending on machine. The problem is to find optimal solution, i.e. that one with the minimal makespan.

Problem overview
-------------------------
The problem is P-hard only for the case when M = 2. Then it's exactly solved by the Johnson's algorithm. The work is dedicated to the case when M > 2, and, moreover, order of jobs on machines could differ. There are several works on the problem, but most of them solves permutation problem, when order on machines remains the same. Our goal is to find an evolutionary approach to the general flow shop problem. 

Chromosome representation
-------------------------
TBD

In [1]:
from collections import deque
from typing import List, Deque
from job import Job
from functools import cmp_to_key

def f2_cmax_johnson_solver(jobs: List[Job]) -> List[Job]:
    """
    Solver for the "F2 || C_max" task introduced in the following article:
    
    S.M. Johnson. Optimal two-and-three-stage production schedules with set-up times included.
    Naval Research Logistic Quaterly, 1:61–68, 1954
    
    link: https://www.rand.org/content/dam/rand/pubs/papers/2008/P402.pdf
    
    :param jobs: list of jobs to optimize schedule
    :returns: jobs ordered to minimize the makespan. As for F2 an order is equal for both stages, list is 1xN
    """
    # Validate input
    for job in jobs:
        if len(job.actions) != 2:
            raise ValueError("Johnson solver is able to solve F2||C_max only. "
                             f"There is a job with {len(job.actions)} actions, so it couldn't be executed as F2 job")

    # Sort by minimal stage time
    def jobs_comparator(lhs: Job, rhs: Job) -> int:
        return min(lhs.actions) - min(rhs.actions)

    sorted_jobs = sorted(jobs, key=cmp_to_key(jobs_comparator))
    first_stage_jobs: Deque[Job] = deque()
    second_stage_jobs: Deque[Job] = deque()

    for job in sorted_jobs:
        if job.actions[0] < job.actions[1]:
            first_stage_jobs.append(job)
        else:
            second_stage_jobs.appendleft(job)

    return list(first_stage_jobs + second_stage_jobs)

In [2]:
from random import randrange

randomJobs = [Job(i + 1, [randrange(1, 5), randrange(1, 5)]) for i in range(20)]
print(*randomJobs, sep="\n")

Job(job_id=1, actions=[3, 2])
Job(job_id=2, actions=[3, 2])
Job(job_id=3, actions=[3, 2])
Job(job_id=4, actions=[1, 4])
Job(job_id=5, actions=[4, 2])
Job(job_id=6, actions=[2, 3])
Job(job_id=7, actions=[1, 3])
Job(job_id=8, actions=[1, 2])
Job(job_id=9, actions=[3, 4])
Job(job_id=10, actions=[3, 3])
Job(job_id=11, actions=[1, 4])
Job(job_id=12, actions=[1, 4])
Job(job_id=13, actions=[4, 3])
Job(job_id=14, actions=[2, 4])
Job(job_id=15, actions=[3, 1])
Job(job_id=16, actions=[3, 1])
Job(job_id=17, actions=[2, 2])
Job(job_id=18, actions=[2, 3])
Job(job_id=19, actions=[4, 2])
Job(job_id=20, actions=[4, 4])


In [3]:
from schedule import order_to_schedule

order = f2_cmax_johnson_solver(randomJobs)
schedule = order_to_schedule([order, order])
print(schedule)

########################################################
478  6 1418 9  20  13 10  19 17 5   3  2  1 16 15 
  4   7 8  11  12  6  14 18  9   20 13 10 19175 3 2 1   
########################################################
Makespan: 56


In [5]:
from main import one_run
from data import load_data

jobs_data = load_data("data2x20.pkl")
runFitness = one_run(jobs_data)
print(runFitness)

[71, 71, 70, 70, 70, 70, 70, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 67, 67, 67, 66, 66, 64, 64, 64, 64, 63]
